In [1]:
def outer():
    x = 'python'
    def inner():
        print(x)
    return inner

In [2]:
fn = outer()

fn.__code__.co_freevars

('x',)

In [3]:
# This property states that fn points to a cell object, and that
# cell object points to a string object (x)
fn.__closure__

(<cell at 0x7f6418ba9730: str object at 0x7f6435a10d30>,)

In [4]:
def outer():
    x = [1, 2, 3]
    print(hex(id(x)))
    def inner():
        y = x
        print(hex(id(y)))
    return inner

In [5]:
fn = outer()

0x7f6419627880


In [6]:
# We can see that fn() uses the same reference for x that was contained in outer(),
# even though the outer function is already out of scope
fn()

0x7f6419627880


In [20]:
def outer():
    count = 0
    def inc():
        nonlocal count
        count += 1
        return count
    return inc

In [21]:
fn = outer()
fn.__closure__

(<cell at 0x7f6418ba93a0: int object at 0x7f6438716910>,)

In [22]:
# we can see that when we modify the free variable 'count' by calling inc()
# the cell that inc references remains the same, but the int object the cell references
# has changed (since ints are immutable, we created a new object by incrementing)
print(fn())
fn.__closure__

1


(<cell at 0x7f6418ba93a0: int object at 0x7f6438716930>,)

In [23]:
def outer():
    count = 0
    def inc1():
        nonlocal count
        count += 1
        return count
    
    def inc2():
        nonlocal count
        count += 1
        return count
    
    return inc1, inc2

In [24]:
fn1, fn2 = outer()

In [27]:
fn1.__closure__, fn2.__closure__

((<cell at 0x7f6418ec0790: int object at 0x7f6438716910>,),
 (<cell at 0x7f6418ec0790: int object at 0x7f6438716910>,))

In [28]:
fn1()

1

In [29]:
fn1.__closure__, fn2.__closure__

((<cell at 0x7f6418ec0790: int object at 0x7f6438716930>,),
 (<cell at 0x7f6418ec0790: int object at 0x7f6438716930>,))

In [30]:
def pow(n):
    def inner(x):
        return x ** n
    
    return inner

In [31]:
sq = pow(2)

In [32]:
sq.__closure__

(<cell at 0x7f6418ec0af0: int object at 0x7f6438716950>,)

In [34]:
cube = pow(3)

In [37]:
# Although both sq() and cube() were created by the same function, they reference different
# cells. A new scope is created for each function call of pow()
cube.__closure__

(<cell at 0x7f6418ec0b50: int object at 0x7f6438716970>,)

In [38]:
def adder(n):
    def inner(x):
        return x + n
    
    return inner

In [39]:
add_1 = adder(1)
add_2 = adder(2)

In [40]:
add_1.__closure__

(<cell at 0x7f6418d367f0: int object at 0x7f6438716930>,)

In [41]:
add_2.__closure__

(<cell at 0x7f6418d368e0: int object at 0x7f6438716950>,)

In [44]:
# Beware, since the reference for n will be the same for all appended lambdas,
# therefore all 'adder' functions will use n==2
def create_adders():
    adders = list()
    for n in range(1, 3):
        adders.append(lambda x: x + n)
    return adders

In [45]:
adders = create_adders()

In [47]:
# Here we can see both adders refference the same cell and the same int object
adders[0].__closure__, adders[1].__closure__

((<cell at 0x7f6418eb6490: int object at 0x7f6438716950>,),
 (<cell at 0x7f6418eb6490: int object at 0x7f6438716950>,))

In [48]:
# We can remedy this issue by setting a default value of y==n, default values are evaluated
# at creation, therefore each adder function will correctly reference n
def create_adders():
    adders = list()
    for n in range(1, 3):
        adders.append(lambda x, y=n: x + y)
    return adders

In [60]:
class Averager:
    def __init__(self):
        self.numbers = list()
        
    def add(self, number):
        self.numbers.append(number)
        total = sum(self.numbers)
        count = len(self.numbers)
        return total / count

In [61]:
a = Averager()

In [62]:
a.add(10)

10.0

In [63]:
a.add(100)

55.0

In [64]:
def averager():
    numbers = list()
    def add(number):
        numbers.append(number)
        total = sum(numbers)
        count = len(numbers)
        return total / count
    return add

In [65]:
add = averager()

In [66]:
add(10)

10.0

In [67]:
add(100)

55.0

In [68]:
def averager():
    total, count = 0, 0
    def add(number):
        nonlocal total, count
        total = total + number
        count += 1
        return total / count
    return add

In [69]:
add = averager()
add(10)
add(100)

55.0

In [70]:
from time import perf_counter

In [76]:
class Timer:
    def __init__(self):
        self.start = perf_counter()
        
    def __call__(self):
        return perf_counter() - self.start

In [77]:
t1 = Timer()

In [78]:
t1()

0.32161684799939394

In [79]:
t1()

4.088377288004267

In [83]:
def timer():
    start = perf_counter()
    def poll():
        return perf_counter() - start
    return poll

In [84]:
t2 = timer()

In [85]:
t2()

0.4114427849999629

In [86]:
t2()

3.882288751003216

In [87]:
def counter(initial=0):
    def inc(increment=1):
        nonlocal initial
        initial += increment
        return initial
    return inc

In [88]:
c1 = counter()

In [89]:
c1()

1

In [91]:
c1()

2

In [99]:
c = dict()
def fn_counter(fn, count_dict):
    cnt = 0
    def inner(*args, **kwargs):
        nonlocal cnt
        cnt += 1
        count_dict[fn.__name__] = cnt
        return fn(*args, **kwargs)
    return inner

In [100]:
def add(a, b): return a + b
def mult(a, b): return a * b

In [101]:
add_and_count = fn_counter(add, c)

In [102]:
add_and_count(5, 10)

15

In [103]:
mult_and_count = fn_counter(mult, c)
mult_and_count(10, 10)

100

In [104]:
c

{'add': 1, 'mult': 1}

In [105]:
def fac(n):
    product = 1
    for i in range(2, n+1):
        product *= i
    return product

In [107]:
fac = fn_counter(fac, c)
fac(2)
fac(5)
fac(10)

3628800

In [108]:
c

{'add': 1, 'mult': 1, 'fac': 3}